In [1]:
from astropy.table import join
from astropy.table import Table

from CRTS_paper_modules import update_progress

import numpy as np
import matplotlib.pyplot as plt 
from os import listdir
import pandas as pd 


# raw LC info on aggregates, i.e. grouped-by SDSS ra, to find stats on individual objects :
#          mean magnitude, mean error,   mjd_span = max(mjd)-min(mjd), lc_length = np.count_nonzero()
# 
# and added processed day-averaged LC info : 
#          proc_mjd_span = max(mjd)-min(mjd) , proc_lc_length, avg_N_per_day  = average number of epochs per day 
#          avg_day_mag = average of day-averaged magnitudes , avg_day_err = average of day-averaged weighted error 


## Match QSO catalogs... 

Okay : read in PTF info, combining statistics  on raw and day-averaged lightcurves :  

In [13]:

address  = '../data_products/PTF_qso_agg_info_2753_lc_merged.dat' 
ptf_qso_table = Table.read(address,format='ascii' )

ptf_qso_table.colnames

['ra_sdss',
 'dec_sdss',
 'oid',
 'avg_err',
 'avg_mag',
 'mjd_span',
 'lc_length',
 'avg_day_mag',
 'proc_mjd_span',
 'avg_day_err',
 'avg_N_per_day',
 'proc_lc_length']

Now, read in SDSS info about quasars : 

In [8]:
from B_sf_CRTS_SDSS_matching_NEW import load_sdss_qso

In [9]:
sdss_qso_info = load_sdss_qso()

zipping quasars...
I read in data for 9258 SDSS quasars 
From catalog ../catalogs_SDSS/DB_QSO_S82.dat


In [11]:
# make an Astropy table from these stats
sdss_qso_table = Table(sdss_qso_info)

In [17]:
# Even though I explicitly used sdss ra, there must be some floating point imprecision
# just match the two catalogs using astropy 


# they are both in decimal degrees, which is a joy :) 

np.sum(np.in1d(ptf_qso_table['ra_sdss'], sdss_qso_table['ra']))

78

In [26]:
from B_sf_CRTS_SDSS_matching_NEW import match_catalogs 

SDSS_matching_rows, matched_radius= match_catalogs(cat1_ra=ptf_qso_table['ra_sdss'],  
                                                   cat1_dec=ptf_qso_table['dec_sdss'], 
                                                   cat2_ra= sdss_qso_table['ra'], 
                                                   cat2_dec=sdss_qso_table['dec']) 
match_angle_deg = matched_radius.value

In [88]:
match_angle_arcsec = match_angle_deg * 3600
# remove one object that does not have a match within 2 arcsec...
# But I wonder why, since all the PTF QSO I got were from SDSS...
# But since my query was positional, maybe for this one QSO
# PTF did not find a nearby object, 
# and that's why it gave something so distant... Even though my matching radius was 
# 2 arcsec?  I'd expect that if something is separated by more than 2 arcsec 
# I'd not get a match.... 


In [75]:
# add columns from the SDSS rows matching to PTF 
ptf_qso_table.add_columns(sdss_qso_table[SDSS_matching_rows].columns.values())

In [61]:
from astropy.table import Column
match_radius_column = Column(match_angle_arcsec, name='match_radius_arcsec')
ptf_qso_table.add_column(match_radius_column)

In [62]:
# show that we did a good job of matching PTF and SDSS catalogs : 
ptf_qso_table[:6][['oid', 'ra','ra_sdss', 'dec', 'dec_sdss', 'match_radius_arcsec']]

oid,ra,ra_sdss,dec,dec_sdss,match_radius_arcsec
float64,float64,float64,float64,float64,float64
2.69710600005e+13,0.049842,0.049846,0.040372,0.040389,0.0628712962885
2.69710600019e+13,0.07453,0.074522,0.436845,0.436856,0.0489648014736
2.69710600037e+13,0.225083,0.225083,0.917519,0.917532,0.0468000000011
2.59210600025e+13,0.331015,0.331011,-1.209549,-1.209551,0.0160968196225
2.69710100004e+13,0.912976,0.91298,1.172123,1.172133,0.0387720676539
2.59210800037e+13,1.148969,1.148971,-1.221839,-1.221851,0.0437956211117


In [63]:
# Save that PTF - SDSS matched catalog... 
ptf_qso_table.write('../data_products/PTF_SDSS_cross_matched_qso_DB9_catalog.dat', format='ascii', overwrite='True')

## Match stars catalogs... 

In [66]:
address  = '../data_products/PTF_stars_agg_info_15714_lc_merged.dat'
ptf_stars_table = Table.read(address,format='ascii' )

In [67]:
from B_sf_CRTS_SDSS_matching_NEW import load_sdss_stars
sdss_stars_info = load_sdss_stars()

Zipping the stars...
I read in data for 1006849 SDSS stars


In [77]:
# Could use that as well to be faster,  because I only have the PTF data for CRTS subset of stars, 
# but using the original SDSS catalog ensures that I am following identical procedure for PTF...

#import CRTS_paper_modules as mod 
#cols2 , star_cat= mod.get_stars_catalog()

In [71]:
# make an Astropy table from these stats
sdss_stars_table = Table(sdss_stars_info)

In [72]:
# Even though I explicitly used sdss ra, there must be some floating point imprecision
# just match the two catalogs using astropy 


# they are both in decimal degrees, which is a joy :) 

np.sum(np.in1d(ptf_stars_table['ra_sdss'], sdss_stars_table['ra']))

42

In [73]:
from B_sf_CRTS_SDSS_matching_NEW import match_catalogs 

SDSS_matching_rows, matched_radius= match_catalogs(cat1_ra=ptf_stars_table['ra_sdss'],  
                                                   cat1_dec=ptf_stars_table['dec_sdss'], 
                                                   cat2_ra= sdss_stars_table['ra'], 
                                                   cat2_dec=sdss_stars_table['dec']) 
match_angle_deg = np.array([a.value  for a in matched_radius])

In [74]:
match_angle_arcsec = match_angle_deg * 3600
# remove one object that does not have a match within 2 arcsec...
# But I wonder why, since all the PTF QSO I got were from SDSS...
# But since my query was positional, maybe for this one QSO
# PTF did not find a nearby object, 
# and that's why it gave something so distant... Even though my matching radius was 
# 2 arcsec?  I'd expect that if something is separated by more than 2 arcsec 
# I'd not get a match.... 


# Check that object! 

In [ ]:
# add columns from the SDSS rows matching to PTF 
ptf_stars_table.add_columns(sdss_stars_table[SDSS_matching_rows].columns.values())

In [76]:
# make an AstroPy Column from the match_angle , and add it to the combined catalog...

from astropy.table import Column
match_radius_column = Column(match_angle_arcsec, name='match_radius_arcsec')
ptf_stars_table.add_column(match_radius_column)

In [78]:
# show that we did a good job of matching PTF and SDSS catalogs : 
ptf_stars_table[:6][['oid', 'ra','ra_sdss', 'dec', 'dec_sdss', 'match_radius_arcsec']]

oid,ra,ra_sdss,dec,dec_sdss,match_radius_arcsec
float64,float64,float64,float64,float64,float64
2.5921e+13,0.054511,0.054504,-1.1499029,-1.149916,0.053468213405
2.69710600012e+13,0.078358,0.078347,0.235094,0.235053,0.152819807627
2.69710600032e+13,0.093134,0.093096,0.751608,0.751601,0.139090114011
2.69710600015e+13,0.132512,0.132509,0.336862,0.336847,0.0550693741422
2.6971060004e+13,0.262431,0.262434,0.965064,0.965061,0.0152724232486
2.69710600038e+13,0.282395,0.28241,1.011578,1.011579,0.054111469592


In [79]:
# Save that PTF - SDSS matched catalog... 
ptf_stars_table.write('../data_products/PTF_SDSS_cross_matched_stars_catalog.dat', format='ascii', overwrite='True')